In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_dataset = np.genfromtxt('resources/train.csv', dtype=np.float32, delimiter=',')
train_targets = train_dataset[:,-1]
test_dataset = np.genfromtxt('resources/test.csv', dtype=np.float32, delimiter=',')
test_targets = test_dataset[:,-1]
combined_dataset=np.concatenate((train_dataset[:,:-1], test_dataset[:,:-1]))

In [4]:
numeric_indices = []
for i in range(combined_dataset.shape[1]):
    n_unique = len(np.unique(combined_dataset[:, i]))
    numeric_indices.append(n_unique > 2)
print (len(numeric_indices))
numeric_indices = np.array(numeric_indices, dtype=np.bool)   

122


In [5]:
combined_dataset[:, numeric_indices] = StandardScaler().fit_transform(
                                        combined_dataset[:, numeric_indices]
                                        )

In [6]:
batch_size = 1500
num_feature = 122
num_labels = 5
graph = tf.Graph()
with graph.as_default():

    tf_noisy_train_dataset = tf.placeholder(tf.float32, shape=(None, num_feature))
    tf_train_dataset = tf.placeholder(tf.float32, shape=(None, num_feature))
    tf_test_dataset = tf.constant(combined_dataset)
    
    n_hidden_1 = 200
#     n_hidden_2 = 100
#     n_hidden_3 = 500
    n_input = num_feature

    wh1 = tf.Variable(tf.truncated_normal([n_input, n_hidden_1], dtype=tf.float32))
#     wh2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], dtype=tf.float32))
#     wh3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3], dtype=tf.float32))
#     wo = tf.Variable(tf.zeros([n_hidden_1, n_input], dtype=tf.float32))
#     wo.initialized_value = tf.transpose(wh1)
    b1 = tf.Variable(tf.truncated_normal([n_hidden_1], dtype=tf.float32))
#     b2 = tf.Variable(tf.truncated_normal([n_hidden_2], dtype=tf.float32))
#     b3 = tf.Variable(tf.truncated_normal([n_hidden_3], dtype=tf.float32))
    bo = tf.Variable(tf.truncated_normal([n_input], dtype=tf.float32))
    layer_1 = tf.add(tf.matmul(tf_noisy_train_dataset, wh1), b1)
    def compute_logit():
#         layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, wh2), b2))
#         layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, wh3), b3))
        return tf.matmul(layer_1, tf.transpose(wh1)) + bo 

    
    logits = compute_logit()
    difference = tf.sub(tf_train_dataset, logits) 
    loss = tf.sqrt(tf.reduce_mean(tf.square(difference)))# +  0.0001 * (tf.nn.l2_loss(wh1) + tf.nn.l2_loss(wo))
#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_dataset))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    train_prediction = compute_logit(tf_train_dataset)

In [7]:
num_steps = 10000
preds = []
randomized_dataset = combined_dataset.copy()
np.random.shuffle(randomized_dataset)
new_input=[]
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (randomized_dataset.shape[0] - batch_size)

        batch_data = randomized_dataset[offset:(offset + batch_size), :]
        noisy_batch_data = batch_data.copy()
        for i, point in enumerate(noisy_batch_data):
            noisy_batch_data[i, :] = point + np.random.normal(0, 1, noisy_batch_data.shape[1])
        feed_dict = {tf_train_dataset : batch_data, tf_noisy_train_dataset: noisy_batch_data}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step", step, ":", l)
#             print "SKLEARN loss", np.sqrt(mean_squared_error(predictions, batch_data))
    new_input=layer_1.eval(feed_dict = {tf_train_dataset :combined_dataset})

Initialized
Minibatch loss at step 0 : 99.9976
Minibatch loss at step 500 : 28.567
Minibatch loss at step 1000 : 23.1939
Minibatch loss at step 1500 : 22.3431
Minibatch loss at step 2000 : 6.3733
Minibatch loss at step 2500 : 4.06738
Minibatch loss at step 3000 : 25.2494
Minibatch loss at step 3500 : 1.68547
Minibatch loss at step 4000 : 0.943027
Minibatch loss at step 4500 : 0.633994
Minibatch loss at step 5000 : 0.338074
Minibatch loss at step 5500 : 0.425005
Minibatch loss at step 6000 : 0.278197
Minibatch loss at step 6500 : 0.260194
Minibatch loss at step 7000 : 0.157855
Minibatch loss at step 7500 : 0.143235
Minibatch loss at step 8000 : 0.117742
Minibatch loss at step 8500 : 0.116883
Minibatch loss at step 9000 : 0.122437
Minibatch loss at step 9500 : 0.0956483


In [10]:
#saving new train & test data 
from collections import Counter
new_train_data=new_input[:train_dataset.shape[0]]
new_train_data=np.c_[new_train_data,train_targets]
new_test_data=new_input[train_dataset.shape[0]:]
new_test_data=np.c_[new_test_data, test_targets]
print(new_train_data.shape)
np.savez('out/new_input.npz',train=new_train_data, test=new_test_data)

(125973, 201)


In [11]:
model = LogisticRegression(C=1,class_weight='balanced',n_jobs=-1)

In [12]:
print (cross_val_score(model, new_train_data[:, :-1], train_targets, cv=3, scoring='precision_macro').mean())

0.732072046713


In [5]:
new_train_data = np.load('out/new_input.npz')['train']
new_test_data = np.load('out/new_input.npz')['test']

In [22]:
model.fit(new_train_data[:, :-1], train_targets)

LogisticRegression(C=1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
preds = model.predict(new_test_data[:, :-1])
# np.savez('out/preds.npz',preds=preds)

In [24]:
print(accuracy_score(preds, test_targets))

0.756919801278


In [25]:
print(confusion_matrix(test_targets, preds))
print(precision_score(test_targets, preds, average='macro'))

[[6056 1344   58    0    0]
 [ 421 8978  291   15    6]
 [ 120  546 1669   86    0]
 [   2 2196    8  342    6]
 [   5  362    1   13   19]]
0.754402008597
